# Collocate MUR SST with SMAP salinity & Saildrone
-this reads in the MUR SST from AWS PODAAC collocates it with all Saildrone cruises that have already been collocated with SMAP salinity
- The code stops running about every 12 files. It doesn't break but it starts returning nan for SSTs that is collocating


# Read in All Saildrone cruises downloaded from https://data.saildrone.com/data/sets
- 2017 onwards, note that earlier data is going to lack insruments and be poorer data quality in general
- For this code I want to develop a routine that reads in all the different datasets and creates a standardized set
- It may work best to first read each of the files individually into a dictionary 
- then go through each dataset finding all variable names
- I decided to put all SST into TEMP_CTD_MEAN and same for Salinity so there is a single variable name
- this still preserves all the dataset information

# now gridded no repeat

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import xarray as xr
import cartopy.crs as ccrs
from scipy import spatial
#sys.path.append('/home/jovyan/shared/users/cgentemann/notebooks/salinity/subroutines/')
#from read_routines import read_all_usv, read_one_usv, add_coll_vars
import warnings
warnings.simplefilter('ignore') # filter some warning messages
from glob import glob


# these libraries help reading cloud data
import fsspec 
import s3fs
import requests
import os

warnings.simplefilter("ignore")  # filter some warning messages
xr.set_options(display_style="html",keep_attrs=True)  # display dataset nicely


In [ ]:
dir_data_pattern = '/home/jovyan/data/sss_collocations_8day_nearest_norepeat/'
dir_out =         '/home/jovyan/data/sss_collocations_8day_nearest_norepeat_mur/'
files = glob(dir_data_pattern+'*.nc')
for ifile,file in enumerate(files):
    ds = xr.open_dataset(file)
    ds.close()
    if any(v=='ob' for v in ds.dims.keys()):
        ds = ds.swap_dims({'ob':'time'})
    #remove any duplicates in time, keep only first value
    _, index = np.unique(ds['time'], return_index=True)
    ds=ds.isel(time=index)
    name = file[57:-3]
    name = name.replace(" ", "_")
    name = name.replace("/", "_")
    if ifile==0:
        data_dict = {name:ds}
    else:
        data_dict[name]=ds
    print(ifile,name)

In [ ]:
for iname,name in enumerate(data_dict):
    fout = dir_out+name+'_20211116.nc'
    ds_usv = xr.open_dataset(fout)
    ds_usv.close()
    print(iname,ds_usv.analysed_sst.mean().data)
    #plt.show()
    #plt.clf()

In [ ]:
import os
import sys
sys.path.append(os.path.join(os.environ['HOME'],'shared','users','lib'))
import ebdpy as ebd

ebd.set_credentials(profile='esip-qhub')

profile = 'esip-qhub'
region = 'us-west-2'
endpoint = f's3.{region}.amazonaws.com'
ebd.set_credentials(profile=profile, region=region, endpoint=endpoint)
worker_max = 30
client,cluster = ebd.start_dask_cluster(profile=profile,worker_max=worker_max, 
                                      region=region, use_existing_cluster=True,
                                      adaptive_scaling=False, wait_for_cluster=False, 
                                      environment='pangeo', worker_profile='Medium Worker', 
                                      propagate_env=True)

In [ ]:
from earthdata import Auth 
auth = Auth().login()

In [ ]:
url = "https://archive.podaac.earthdata.nasa.gov/s3credentials"
response = requests.get(url).json()

In [ ]:
%%time
# set up read
json_consolidated = "s3://esip-qhub-public/nasa/mur/murv41_consolidated_20211011.json"
s_opts = {"requester_pays": True, "skip_instance_cache": True}
r_opts = {"key": response["accessKeyId"],"secret": response["secretAccessKey"],"token": response["sessionToken"],"client_kwargs": {"region_name": "us-west-2"},}
fs = fsspec.filesystem("reference",fo=json_consolidated,
                       ref_storage_args=s_opts,remote_protocol="s3",
                       remote_options=r_opts,simple_templates=True,)
ds_sst = xr.open_dataset(fs.get_mapper(""), decode_times=False, engine="zarr", consolidated=False)
ds_sst

In [ ]:
ds = ds_sst
for iname,name in enumerate(data_dict):
    if iname<87:
        continue
    print(iname,name)
    ds_usv = data_dict[name]
    #create space for new data
    for var in ds_sst:  
        ds_usv[var]=ds_usv.BARO_PRES_MEAN.copy(deep=True)*np.nan
        ds_usv[var].attrs=ds_sst[var].attrs
    ilen = len(ds_usv.time)
    for inc in range(0,ilen,5):
        #print(inc)
        i1,i2 = inc,inc+5
        if i2>ilen:
            i2=ilen-1
        if i1==i2:
            continue
        #print(inc,inc+101)
        sub = ds_usv.isel(time=slice(i1,i2))   
        t1,t2=sub.time.min().data-np.timedelta64(1,'D'),sub.time.max().data+np.timedelta64(1,'D')
        x1,x2=sub.lon.min().data-.15,sub.lon.max().data+.15
        y1,y2=sub.lat.min().data-.15,sub.lat.max().data+.15
        #print(inc,t1,t2,x1,x2,y1,y2)
        ds_sat = ds_sst.sel(time=slice(t1,t2),lat=slice(y1,y2),lon=slice(x1,x2))  
        ds_sat['time']=np.asarray(ds_sat.time.data, "datetime64[ns]") 
        ds_interp = ds_sat.interp(time=sub.time,lat=sub.lat,lon=sub.lon,method='linear',assume_sorted=False) #add saildrone data to interpolated sat data
        #add saildrone data to interpolated sat data
        ds_interp = ds_interp.reset_coords(names={'lat','lon'})
        for var in ds_interp:
            ds_usv[var][i1:i2]=ds_interp[var]        
    #output
    fout = dir_out+name+'_20211116.nc'
    ds_usv.to_netcdf(fout)
    val = ds_usv.analysed_sst.mean().data
    if np.isnan(val):
        print(iname,val)
        print('STOP')
        break
    print('output done, start new')

In [ ]:
ds_usv.to_netcdf(fout)

In [ ]:
ds_interp = ds_sat.interp(time=sub.time).load()
#ds_interp = ds_interp.reset_coords(names={'lat','lon'})
#ds_interp.analysed_sst.plot()
#ds_interp = ds_interp.drop('ob')
ds_interp.analysed_sst[0,:,:].plot()

In [ ]:
ds_sst

In [ ]:
ds_sst.analysed_sst[5000,0:1000,18000:19000].plot()

In [ ]:
ds_sst.analysed_sst[5000,9000,18000]

# tricky bit here, .interp wasn't working
- ds_sat is being read somewhere as "datetime64[us]" rather than "datetime64[ns]"
- this is breaking the interpolation routine which expects "datetime64[ns]"
- solution is to set ds_sat time to "datetime64[ns]"

In [ ]:
ds_sat.time

In [ ]:
data = np.asarray(ds_sat.time.data, "datetime64[ns]") 
ds_sat['time']=data

In [ ]:
tem2 = ds_sat.interp(time=ds_usv.time,lat=ds_usv.lat,lon=ds_usv.lon,method='linear',assume_sorted=False)
#tem2 = ds_sat.sel(time=ds_sat.time[1],method='nearest')#,lat=ds_usv.lat[0],lon=ds_usv.lon[0],method='linear',assume_sorted=False)
#tem2 = ds_sat.sel(time=ds_usv.time[0],tem2 = ds_sat.sel(time=ds_sat.time[1],method='nearest')#,lat=ds_usv.lat[0],lon=ds_usv.lon[0],method='linear',assume_sorted=False)
#tem2 = ds_sat.sel(time=data[0],method='nearest')#,lat=ds_usv.lat[0],lon=ds_usv.lon[0],method='linear',assume_sorted=False)
#lat=ds_usv.lat[0],lon=ds_usv.lon[0],method='nearest')#,method='linear',assume_sorted=False)
tem2.analysed_sst.plot()

In [ ]:
tem2 = ds_sat.sel(time=sub.time,lat=sub.lat,lon=sub.lon,method='nearest') 
tem2.analysed_sst.plot()

# TESTING